## Przykłady

In [2]:
from auth import openai_api_key

In [4]:
# 01_langchain_init

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Inicjalizacja domyślnego modelu, czyli gpt-3.5-turbo
chat = ChatOpenAI(openai_api_key=openai_api_key)

# Wywołanie modelu poprzez przesłanie tablicy wiadomości.
# W tym przypadku to proste przywitanie
message = HumanMessage("Hej, jestem Justynka!")
response = chat.invoke([message])

print(response.content)

Cześć Justynko! Jak się masz? W czym mogę Ci dzisiaj pomóc?


In [25]:
# 02_langchain_format

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# from langchain.context import context

# Treść kontekstu
context = """
Vercel AI SDK to biblioteka typu open-source zaprojektowana, aby pomóc programistom tworzyć interfejsy użytkownika dla konwersacji, streamingu i czatu w JavaScript i TypeScript. SDK wspiera React/Next.js, Svelte/SvelteKit, przy czym wsparcie dla Nuxt/Vue będzie dostępne wkrótce.
Aby zainstalować SDK, wprowadź następujące polecenie w terminalu:
npm install ai
"""

# Zwykle do definiowania promptów warto korzystać z template strings
# Tutaj treści zamknięte w klamrach {} są zastępowane przez LangChain konkretnymi wartościami
system_template = """
As a {role} who answers the questions ultra-concisely using CONTEXT below 
and nothing more and truthfully says "don't know" when the CONTEXT is not enough to give an answer.

context###{context}###
"""

human_template = "{text}"

# Lista wiadomości
messages = [
    ("system", system_template),
    ("human", human_template)
]

# Inicjalizacja promptu
chat_prompt = ChatPromptTemplate.from_messages(messages)

# Faktyczne uzupełnienie szablonów wartościami
formatted_chat_prompt = chat_prompt.format_messages(
    context= context,
    role= "Doświadczony programista JavaScript",
    text= "Co to wyrażenia lambda w Pythonie?",
)

# Inicjalizacja domyślnego modelu, czyli gpt-3.5-turbo
chat = ChatOpenAI(openai_api_key=openai_api_key)

# Wykonanie zapytania do modelu
response =  chat.invoke(formatted_chat_prompt)

print(response.content)

Nie wiem


W przypadku definiowania contextu i systemowej templatki należy używać języka ANGIELSKIEGO. Jeśli robię to po polsku, to odpowiedzi są niezgodne z poleceniem - AI nie używa odpowiedzi "nie wiem". Po zdefiniowaniu wytycznych po angielsku można zadawac pytanie zarówno po angielsku jak i po polsku i odpowiada dobrze

In [18]:
# 03_langchain_stream

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

## Inicjalizacja chatu z włączonym streamingiem
chat = ChatOpenAI(
    openai_api_key=openai_api_key, 
    streaming=True
)

# Wywołanie chatu wraz z funkcją przyjmującą kolejne tokeny składające się na wypowiedź modelu
chunks = []
async for chunk in chat.astream("Cześć, jestem Justynka!"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|C|ze|ść| Just|yn|ko|!| Jak| mog|ę| Ci| dz|isia|j| pom|ó|c|?||

więcej o streamingu tutaj: https://python.langchain.com/docs/expression_language/streaming

In [1]:
# 04_tiktoken

from typing import List, Dict, Optional
from tiktoken import get_encoding

# Definicja typu Message zgodnie z podaną definicją
Message = Dict[str, Optional[str]]

def count_tokens(messages: List[Message], model="gpt-3.5-turbo-0613") -> int:
    encoding = get_encoding("cl100k_base")

    tokens_per_message, tokens_per_name = 0, 0
    if model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k-0613", "gpt-4-0314", "gpt-4-32k-0314", "gpt-4-0613", "gpt-4-32k-0613"]:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4
        tokens_per_name = -1
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return count_tokens(messages, "gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return count_tokens(messages, "gpt-4-0613")
    else:
        raise ValueError(f"num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.")

    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            if value is not None:  # Upewniamy się, że przetwarzamy tylko istniejące wartości
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens


In [4]:
from typing import List
from tiktoken import get_encoding

# Definicja Message zgodnie z wcześniejszym przykładem
Message = dict

# Przykładowa lista wiadomości
messages: List[Message] = [
    {
        "role": "system",
        "content": "Cześć jestem Justynka!",
    }
]

# Wywołanie funkcji countTokens i wydruk wyniku
num = count_tokens(messages, 'gpt-4')  # 11 to przykładowy wynik, tutaj wywołujemy funkcję
print(f"Token Count: {num}")

# Użycie get_encoding do zakodowania treści pierwszej wiadomości
encoding = get_encoding("cl100k_base")
print(f"Token IDs: {encoding.encode(messages[0]['content'])}")


Token Count: 16
Token IDs: [34, 3059, 25898, 13599, 336, 4702, 1910, 4657, 0]


## Zadania praktyczne

In [5]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey

In [6]:
from dotenv import load_dotenv

# Automatycznie wyszukuje plik .env w katalogu bieżącym i załaduje zmienne środowiskowe
load_dotenv()

import os
openai_api_key = os.getenv("OPENAI_API_KEY")

### Moderation

In [42]:
# autoryzacja

task_name = "moderation"

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token:", token)
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token: 8aaef3d04af75f29ce34722db8627efff495a4f8


In [43]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please return 0/1 for each sentence to classify if this is content that requires moderation', 'input': ['azjaci są głupi i brzydcy i nie powinni żyć', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego']}


In [44]:
print(response['input'])

['azjaci są głupi i brzydcy i nie powinni żyć', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego']


In [45]:
from openai import OpenAI
client = OpenAI()

answers = []
for statement in response['input']:
    response = client.moderations.create(input=statement)
    
    output = response.results[0]
    if(output.flagged) == True:
        answers.append(1)
    else:
        answers.append(0)

In [46]:
print(answers)

[1, 1, 1, 0]


In [47]:
# wysłanie odpowiedzi

response = send_answer(token, answers)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


### blogger

In [50]:
from openai import OpenAI
client = OpenAI()

In [51]:
# autoryzacja

task_name = "blogger"

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token


In [52]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please write blog post for the provided outline', 'blog': ['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']}


In [53]:
print(response['blog'])

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


In [54]:
def text_generator(context, prompt):
    completion = client.chat.completions.create(
            model="gpt-4",
            temperature = 0.7,
            messages=[
                    {"role": "system", "content": context},
                    {"role": "user", "content": prompt}
                ]
            )
    return completion.choices[0].message.content

In [55]:
article = []
section_done = []

context = """You are an assistant to create articles for the website. You don't create the whole article right away, but the user sends you the headline of the section to write, and you create it.In addition, the user sends you a list of headings from the article that have already been written. Don't duplicate the information they focused on. Some tips you should follow:
    - focus on creating content that is valuable and well thought out;
    - avoid plagiarism and be creative. Create unique content that will make your text stand out;
    - learn the basics of search engine optimization (SEO) and skillfully use keywords to increase your content's visibility in search results;
    - when writing, be clear and concise. Avoid unnecessary words and complex sentence constructions that can put off readers;
    - use bullets and highlighting to make your content easier to read and navigate. Good structure helps readers quickly find what they are looking for;
    - encourage interaction through questions or calls to action; 
    - use a friendly, conversational tone;"""

for section in response['blog']:
    prompt = f"Stwórz treść sekcji: {section}. Rozwinąłeś już wcześniej takie sekcje: {section_done}."

    section_text = text_generator(context, prompt)
    section_done.append(section)
    article.append(section_text)

In [56]:
# wysłanie odpowiedzi

response = send_answer(token, article)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
